In [36]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
json_prepared = dataiku.Dataset("Json_prepared")
df = json_prepared.get_dataframe()




In [37]:
import math

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import Sequence
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import numpy as np
import pandas as pd
import time

import os

In [38]:
df.head()

,Timestamp,date,broker-102/,broker-104/,broker-106/,broker-108/,broker-110/,zook-102/,zook-104/,broker-102/data,broker-104/data,broker-106/data,broker-108/data,broker-110/data,zook-102/data,zook-104/data,broker-101/,broker-103/,broker-105/,broker-107/,broker-109/,zook-101/,zook-103/,zook-105/,broker-101/data,broker-103/data,broker-105/data,broker-107/data,broker-109/data,zook-101/data,zook-103/data,zook-105/data
0,1624601700,2021-06-25 06:15:00+00:00,16.318256,14.317784,15.499574,13.478442,10.100069,3.457606,3.497103,85.198139,74.491702,70.937154,73.322070,88.858424,0.278296,0.216338,15.759843,15.526538,10.510089,13.464054,9.834396,3.489473,3.470773,3.176120,81.194161,76.236908,73.175427,72.768512,90.265277,0.251977,0.236909,0.245222
1,1624602600,2021-06-25 06:30:00+00:00,16.201163,14.317455,15.596986,13.478452,10.100073,3.457596,3.498734,85.195519,74.540221,70.971025,73.390322,88.953916,0.278296,0.216338,15.798915,15.604688,10.588215,13.542199,9.893027,3.469877,3.470783,3.176225,81.216755,76.227287,73.233565,72.715046,90.343407,0.251977,0.236909,0.245222
2,1624603500,2021-06-25 06:45:00+00:00,16.240144,14.200271,15.479296,13.478466,10.021952,3.457639,3.498534,85.368338,74.687954,71.060131,73.450625,89.056802,0.278296,0.216338,15.875768,15.526571,10.480893,13.464078,9.971639,3.489372,3.470907,3.156653,81.287027,76.350551,73.217323,72.744789,90.583015,0.251977,0.236909,0.245222
3,1624604400,2021-06-25 07:00:00+00:00,16.166954,14.239343,15.479368,13.351567,10.002428,3.458865,3.498543,85.503684,74.622017,71.028285,73.530478,89.180835,0.278296,0.216338,15.875782,15.487523,10.519952,13.464083,9.971339,3.470134,3.470964,3.176167,81.417682,76.318796,73.396165,72.921757,90.711837,0.251977,0.236909,0.245222
4,1624605300,2021-06-25 07:15:00+00:00,16.240354,14.317498,15.597000,13.405028,10.002437,3.497909,3.499302,85.652764,74.762854,71.062395,73.475029,89.232812,0.278296,0.216338,15.758903,15.458370,10.446685,13.542228,9.893218,3.470191,3.472156,3.176158,81.569016,76.359671,73.456350,72.810311,90.751683,0.251977,0.236909,0.245222


In [39]:
del df['Timestamp']

In [40]:

# Split into training, validation and test datasets.
# Since it's timeseries we should do it by date.
test_cutoff_date = df['date'].max() - timedelta(days=7)

test_df = df[df['date'] > test_cutoff_date]
train_df = df[df['date'] <= test_cutoff_date]

#check out the datasets
print('Test dates: {} to {}'.format(test_df['date'].min(), test_df['date'].max()))
print('Train dates: {} to {}'.format(train_df['date'].min(), train_df['date'].max()))

Test dates: 2021-07-29 13:15:00+00:00 to 2021-08-05 13:00:00+00:00
Train dates: 2021-06-25 06:15:00+00:00 to 2021-07-29 13:00:00+00:00


In [41]:
train_df  = train_df.set_index('date')
test_df = test_df.set_index('date')

In [42]:
train_df.head()

,broker-102/,broker-104/,broker-106/,broker-108/,broker-110/,zook-102/,zook-104/,broker-102/data,broker-104/data,broker-106/data,broker-108/data,broker-110/data,zook-102/data,zook-104/data,broker-101/,broker-103/,broker-105/,broker-107/,broker-109/,zook-101/,zook-103/,zook-105/,broker-101/data,broker-103/data,broker-105/data,broker-107/data,broker-109/data,zook-101/data,zook-103/data,zook-105/data
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-06-25 06:15:00+00:00,16.318256,14.317784,15.499574,13.478442,10.100069,3.457606,3.497103,85.198139,74.491702,70.937154,73.322070,88.858424,0.278296,0.216338,15.759843,15.526538,10.510089,13.464054,9.834396,3.489473,3.470773,3.176120,81.194161,76.236908,73.175427,72.768512,90.265277,0.251977,0.236909,0.245222
2021-06-25 06:30:00+00:00,16.201163,14.317455,15.596986,13.478452,10.100073,3.457596,3.498734,85.195519,74.540221,70.971025,73.390322,88.953916,0.278296,0.216338,15.798915,15.604688,10.588215,13.542199,9.893027,3.469877,3.470783,3.176225,81.216755,76.227287,73.233565,72.715046,90.343407,0.251977,0.236909,0.245222
2021-06-25 06:45:00+00:00,16.240144,14.200271,15.479296,13.478466,10.021952,3.457639,3.498534,85.368338,74.687954,71.060131,73.450625,89.056802,0.278296,0.216338,15.875768,15.526571,10.480893,13.464078,9.971639,3.489372,3.470907,3.156653,81.287027,76.350551,73.217323,72.744789,90.583015,0.251977,0.236909,0.245222
2021-06-25 07:00:00+00:00,16.166954,14.239343,15.479368,13.351567,10.002428,3.458865,3.498543,85.503684,74.622017,71.028285,73.530478,89.180835,0.278296,0.216338,15.875782,15.487523,10.519952,13.464083,9.971339,3.470134,3.470964,3.176167,81.417682,76.318796,73.396165,72.921757,90.711837,0.251977,0.236909,0.245222
2021-06-25 07:15:00+00:00,16.240354,14.317498,15.597000,13.405028,10.002437,3.497909,3.499302,85.652764,74.762854,71.062395,73.475029,89.232812,0.278296,0.216338,15.758903,15.458370,10.446685,13.542228,9.893218,3.470191,3.472156,3.176158,81.569016,76.359671,73.456350,72.810311,90.751683,0.251977,0.236909,0.245222


In [43]:
train = train_df
scalers={}
for i in train_df.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    train[i]=s_s
test = test_df
for i in test_df.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    test[i]=s_s

In [44]:
def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations  
  #
  # n_future ==> no of future observations 
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

In [45]:
n_past = 10
n_future = 5 
n_features = 30

In [46]:
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))


In [47]:
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]

#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])

#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)

#
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

#
model_e1d1.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


/opt/dataiku/code-env/lib64/python3.6/site-packages/tensorflow_core/python/framework/tensor_util.py:521: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10, 30)]     0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 100), (None, 52400       input_1[0][0]                    
__________________________________________________________________________________________________
repeat_vector (RepeatVector)    (None, 5, 100)       0           lstm[0][0]                       
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 5, 100)       80400       repeat_vector[0][0]              
                                                                 lstm[0][1]                   

In [48]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e1d1.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e1d1=model_e1d1.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=0,callbacks=[reduce_lr])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/opt/dataiku/code-env/lib64/python3.6/site-packages/tensorflow_core/python/framework/tensor_util.py:521: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()
/opt/dataiku/code-env/lib64/python3.6/site-packages/tensorflow_core/python/framework/tensor_util.py:521: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()


In [49]:
pred_e1d1=model_e1d1.predict(X_test)

In [54]:
for index,i in enumerate(train_df.columns):
    scaler = scalers['scaler_'+i]
    pred_e1d1[:,:,index]=scaler.inverse_transform(pred_e1d1[:,:,index])
    y_train[:,:,index]=scaler.inverse_transform(y_train[:,:,index])
    y_test[:,:,index]=scaler.inverse_transform(y_test[:,:,index])

In [57]:
y_test

array([[[13.73975085, 12.34896357, 13.48752729, ...,  0.28861596,
          0.24067233,  0.21813935],
        [13.81945537, 12.19840105, 13.34242053, ...,  0.28868466,
          0.24073339,  0.21819278],
        [13.70270993, 12.3528217 , 13.35214455, ...,  0.28869993,
          0.24075629,  0.21822331],
        [13.74617948, 12.34994121, 13.48782297, ...,  0.28869993,
          0.24075629,  0.21822331],
        [13.66709971, 12.35417133, 13.34188163, ...,  0.28869993,
          0.24075629,  0.21822331]],

       [[13.81945537, 12.19840105, 13.34242053, ...,  0.28868466,
          0.24073339,  0.21819278],
        [13.70270993, 12.3528217 , 13.35214455, ...,  0.28869993,
          0.24075629,  0.21822331],
        [13.74617948, 12.34994121, 13.48782297, ...,  0.28869993,
          0.24075629,  0.21822331],
        [13.66709971, 12.35417133, 13.34188163, ...,  0.28869993,
          0.24075629,  0.21822331],
        [13.74318931, 12.35760979, 13.43049466, ...,  0.28869993,
          0.24

In [56]:
pred_e1d1.shape

(658, 5, 30)

In [59]:
handle = dataiku.Folder("models")
path 

In [60]:
from keras.models import load_model 
model_e1d1.save('model.h5')

Using TensorFlow backend.


In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

predictions_df = json_prepared_df # For this sample code, simply copy input to output


# Write recipe outputs
predictions = dataiku.Dataset("predictions")
predictions.write_with_schema(predictions_df)